In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
# from imgaug import augmenters as ia
import pandas as pd
import cv2,pickle
from tensorflow.keras import models,layers
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split as tts

In [2]:
fx=open("./X.pkl",'rb')
X=pickle.load(fx)


In [3]:
fy=open("./Y_binary.pkl",'rb')
Y=pickle.load(fy)


In [4]:
NY=[]
# ['angry', 'fear', 'happy', 'neutral', 'sad', 'surprise']
for y in Y:
    if y == 0:
        NY.append([1,0,0,0,0,0])
    elif y == 1:
        NY.append([0,1,0,0,0,0])
    elif y == 2:
        NY.append([0,0,1,0,0,0])
    elif y == 3:
        NY.append([0,0,0,1,0,0])
    elif y == 4:
        NY.append([0,0,0,0,1,0])
    elif y == 5:
        NY.append([0,0,0,0,0,1])
    
NY=np.array(NY)

In [5]:
X_train,X_test,Y_train,Ytest=tts(X,NY,test_size=0.2,shuffle=True)

In [6]:
X_train.shape 

(22708, 48, 48, 3)

In [13]:
Y_train.shape

(22708, 6)

In [20]:
Model=models.Sequential([
    layers.Conv2D(16,(2,2),input_shape=(48,48,3),activation='relu'),
    layers.Conv2D(32,(2,2),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,(2,2),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(128,(2,2),activation='relu'),
    # layers.MaxPool2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(36,activation='relu'),
    layers.Dense(6,activation='softmax')
    ])

# Model.compile(Adam(learning_rate=0.1),loss='binary_crossentropy',metrics=['accuracy'])
# Model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
Model.compile(Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [21]:
Model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 47, 47, 16)        208       
                                                                 
 conv2d_11 (Conv2D)          (None, 46, 46, 32)        2080      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 22, 22, 64)        8256      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 11, 11, 64)       0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 10, 10, 128)      

In [17]:
from tensorflow.keras import callbacks
model_no=0

Load Pre Model

In [17]:
name=os.listdir("./Models")[-1]
path="./Models/"+name
model_no=int(str(name).split("_")[-1].split(".")[0])
Model.load_weights(path)

In [18]:
class callback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        Model.save_weights(f"./Models1/Model_{(epoch+1)+model_no}.h5")
        return super().on_epoch_end(epoch, logs)

# savemodel=callbacks.ModelCheckpoint(filepath=)

In [ ]:
Model.fit(X_train,Y_train,epochs=50,batch_size=500,callbacks=[callback()])

In [ ]:
Model.evaluate(X_test,Ytest)